# Проект 7 

# Определение перспективного тарифа для компании "Мегалайн"

### Клиентам предлагается два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.

Предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В нашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

## Описание тарифов
### Тариф «Смарт»
- Ежемесячная плата: 550 рублей
- ключено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
- Стоимость услуг сверх тарифного пакета:
 - минута разговора: 3 рубля
 - сообщение: 3 рубля
 - 1 Гб интернет-трафика: 200 рублей

### Тариф «Ультра»
- Ежемесячная плата: 1950 рублей
- Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
- Стоимость услуг сверх тарифного пакета:
 - минута разговора: 1 рубль
 - сообщение: 1 рубль
 - 1 Гб интернет-трафика: 150 рублей

In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats as st
import warnings

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 50)

# Рассмотрим датасеты

In [4]:
data_calls = pd.read_csv('/Users/sergejnitkin/mobile_datasets/calls.csv', parse_dates=['call_date'])
data_calls.head(10)

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000
5,1000_5,2018-11-02,0.00,1000
6,1000_6,2018-10-18,0.00,1000
7,1000_7,2018-08-22,18.31,1000
8,1000_8,2018-09-15,18.44,1000
9,1000_9,2018-08-15,0.00,1000


In [ ]:
data_calls.info()

In [5]:
data_internet = pd.read_csv('/Users/sergejnitkin/mobile_datasets/internet.csv', parse_dates=['session_date'])
data_internet.head(10)

,Unnamed: 0,id,mb_used,session_date,user_id
0,0,1000_0,112.95,2018-11-25,1000
1,1,1000_1,1052.81,2018-09-07,1000
2,2,1000_2,1197.26,2018-06-25,1000
3,3,1000_3,550.27,2018-08-22,1000
4,4,1000_4,302.56,2018-09-24,1000
5,5,1000_5,399.97,2018-10-02,1000
6,6,1000_6,540.08,2018-09-07,1000
7,7,1000_7,415.70,2018-07-11,1000
8,8,1000_8,505.40,2018-10-08,1000
9,9,1000_9,345.54,2018-09-03,1000


In [ ]:
data_internet.info()

In [6]:
data_messages = pd.read_csv('/Users/sergejnitkin/mobile_datasets/messages.csv', parse_dates=['message_date'])
data_messages.head(10)

,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000
5,1000_5,2018-06-20,1000
6,1000_6,2018-11-19,1000
7,1000_7,2018-10-29,1000
8,1000_8,2018-06-25,1000
9,1000_9,2018-12-18,1000


In [ ]:
data_messages.info()

In [7]:
data_tariffs = pd.read_csv('/Users/sergejnitkin/mobile_datasets/tariffs.csv')
data_tariffs

,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


In [8]:
data_users = pd.read_csv('/Users/sergejnitkin/mobile_datasets/users.csv', parse_dates=['reg_date', 'churn_date'])
data_users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaT,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaT,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaT,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaT,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaT,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
...,...,...,...,...,...,...,...,...
495,1495,65,NaT,Иркутск,Авксентий,Фокин,2018-08-28,ultra
496,1496,36,NaT,Вологда,Трифон,Блохин,2018-01-27,smart
497,1497,32,NaT,Челябинск,Каролина,Агеева,2018-10-09,smart
498,1498,68,2018-10-25,Владикавказ,Всеволод,Акимчин,2018-07-19,smart


In [ ]:
data_users.info()

## Проверим данные на задвоения.

In [ ]:
data_calls.duplicated().sum()

In [ ]:
data_internet.duplicated().sum()

In [ ]:
data_messages.duplicated().sum()

In [ ]:
data_users.duplicated().sum()

### Теперь проверим data_users. Там нас по большому счету интересуют значения в колонках tariff и city.

In [ ]:
data_users['city'].value_counts()

In [ ]:
data_users['tariff'].value_counts()

Видим, что в таблице данные заполнены корректно.

### Все данные выглядят корректно, в нашем случае nan имеют право быть, это не ошибки.

## Теперь округлим согласно тарифов значения минут и приведем их к целочисленному формату.

Посмотрим на таблицу с минутами, каждый звонок округляется до минут, поэтому округлим в большую сторону и приведем к целочисленному формату. Кроме звонков равных 0, тк это просто пропущенные звонки.

In [ ]:
data_calls['duration'] = np.ceil(data_calls['duration']).astype('int')
data_calls

In [ ]:
data_calls.info()

## Посчитаем для каждого пользователя:
- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам.

### Для этого сначала добавим в таблицы столбец с месяцем.

In [ ]:
data_calls['month'] = data_calls['call_date'].dt.month
data_calls

In [ ]:
data_internet['month'] = data_internet['session_date'].dt.month
data_internet

In [ ]:
data_messages['month'] = data_messages['message_date'].dt.month
data_messages

### теперь построим сводные таблицы по минутам, мб и сообщениям.

Длительность звонков мы уже округлили, поэтому здесь просто сгруппируем данные по пользователям и месяцам

In [ ]:
calls_per_month = data_calls.groupby(['user_id', 'month']).agg({'duration': ['count', 'sum']}).reset_index()
calls_per_month.columns = ['user_id', 'month', 'calls', 'minutes']
calls_per_month

по такому же принципу сгруппируем данные по сообщениям

In [ ]:
messages_per_month = data_messages.groupby(['user_id', 'month']).agg({'id': 'count'}).reset_index()
messages_per_month.columns = (['user_id', 'month', 'messages'])
messages_per_month

Что касается интернет траффика, то тут сессии не округляются, а трафик округляется до гб в конце месяца. Поэтому сначала сгруппируем данные по пользователям и месяцам, и только потом округлим до гб.

In [ ]:
mb_per_month = data_internet.groupby(['user_id', 'month']).agg({'mb_used': 'sum'}).reset_index()
mb_per_month['mb_used'] = np.ceil(mb_per_month['mb_used'] / 1024).astype('int')
mb_per_month.columns = ['user_id', 'month', 'gb_used']
mb_per_month

## Теперь объединим их в одну таблицу для того, чтобы посчитать перерасходы по пакетам и посчитать выручку по месяцам для каждого пользователя

In [ ]:
data_per_month = calls_per_month.merge(mb_per_month, on=['user_id', 'month'], how='outer')
data_per_month

In [ ]:
data_per_month = data_per_month.merge(messages_per_month, on=['user_id', 'month'], how='outer')
data_per_month.head()

In [ ]:
data_per_month = data_per_month.merge(data_users[['user_id', 'tariff', 'city']], on='user_id', how='outer')
data_per_month

In [ ]:
data_tariffs

Теперь для того, чтобы свести все таблицы надо немного изменить таблицу с тариффами, а именно округлить пакет мб до гб и переименуем столбцы соответственно

In [ ]:
data_tariffs['mb_per_month_included'] = data_tariffs['mb_per_month_included'] / 1024
data_tariffs

In [ ]:
data_tariffs.rename(columns = {'mb_per_month_included': 'gb_per_month_included', 'tariff_name': 'tariff'},
                    inplace=True)
data_tariffs

Теперь объединим таблицы

In [ ]:
data_per_month = data_per_month.merge(data_tariffs, on='tariff', how='outer')
data_per_month

In [ ]:
data_per_month.user_id.nunique()

## Заполним пропуски 0.


In [ ]:
data_per_month = data_per_month.fillna(0)
data_per_month.tail(10)

In [ ]:
data_per_month.info()

Теперь для подсчета выручки, нам надо написать функцию, которая проверит, есть ли расход по каким-либо пакетам, и если есть, то посчитает стоимость перерасхода, согласно тариффа, и прибавит к этому ежемесячную плату.

In [ ]:
def revenue(row):
    revenue_per_minutes = 0
    revenue_per_gb = 0
    revenue_per_messages = 0
    if (row['minutes'] - row['minutes_included']) > 0:
        revenue_per_minutes = (row['minutes'] - row['minutes_included']) * row['rub_per_minute']
    if (row['gb_used'] - row['gb_per_month_included']) > 0:
        revenue_per_gb = (row['gb_used'] - row['gb_per_month_included']) * row['rub_per_gb']
    if (row['messages'] - row['messages_included']) > 0:
        revenue_per_messages = (row['messages'] - row['messages_included']) * row['rub_per_message']
    return row['rub_monthly_fee'] + revenue_per_minutes + revenue_per_gb + revenue_per_messages

In [ ]:
data_per_month['revenue'] = data_per_month.apply(revenue, axis=1)
data_per_month

In [ ]:
# на самом деле, сначала я сделал со строками по перерасходам, потому что так проще проверить себя было

## Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц? 
Посчитаем:
- среднее количество, 
- дисперсию
- стандартное отклонение
- медиану 

Построим гистограммы. 


In [ ]:
sns.histplot(data_per_month, x='minutes', bins=30, hue='tariff').set_title('Minutes usage')
plt.show()

у меня же графики не в матплотлиб, а в сиборн

In [ ]:
data_per_month.pivot_table(index='tariff', values='minutes', aggfunc=['mean', 'median', 'var', 'std'])

### Что касается расхода минут, то на графике мы видим следующее:
- пользователи тарифа Смарт расходуют в среднем 417 минут, распределение близко к нормальному, стандартное отклонение равно 190
- что же касается пользователей тарифа Ультра, то тут расброс гораздо шире - стандартное отклонение здесь уже 317 минут. График скошен вправо, о чем говорит разница между медианой и средним. Ну а в среднем пользователи тарифа расходуют 526 минут.

In [ ]:
sns.histplot(data_per_month, x='gb_used', bins=30, hue='tariff').set_title('Internet')
plt.show()

In [ ]:
data_per_month.pivot_table(index='tariff', values='gb_used', aggfunc=['mean', 'median', 'var', 'std'])

### Теперь посмотрим на то как расходуются гигабайты:
- по расходованию интернет трафика пользователи тарифа Смарт распределились нормально. Основная масса людей тратит от 10 до 20 гб трафика. В среднем пользователи расходуют 16.3гб в месяц
- Пользователи тарифа Ультра тоже имеют нормальное распределение, но имеют более широкую дисперсию. В среднем пользователи расходуют 19.5гб.

In [ ]:
sns.histplot(data_per_month, x='messages', bins=30, hue='tariff').set_title('Messages')
plt.show()

In [ ]:
data_per_month.pivot_table(index='tariff', values='messages', aggfunc=['mean', 'median', 'var', 'std'])

## Сообщения:
- пользователи тарифа Смарт в основном отправляют до 50 сообщений в месяц, на графике мы можем видеть распределение Пуассона. В среднем пользователи отправляют 33.4 сообщений
- пользователи тарифа Ультра отправляют в среднем 49.3 сообщений, в то время как медиана 38 сообщение, это говорит о выбросах, которые мы можем наблюдать на графике.


# Гипотезы 

- Нулевая гипотеза - средняя выручка пользователей тарифов «Ультра» и «Смарт» равна
- Альтернативная гипотеза - средняя выручка пользователей тарифов "Ультра" больше, чем свредняя выручка пользователей тарифа "Смарт" 

In [ ]:
alpha = .05
results_tariffs = st.ttest_ind(data_per_month[data_per_month['tariff'] == 'ultra']['revenue'], 
                      data_per_month[data_per_month['tariff'] == 'smart']['revenue'], equal_var=False)
display(results_tariffs.pvalue)
if (results_tariffs.pvalue / 2) < alpha and (data_per_month[data_per_month['tariff'] == 'ultra']['revenue'].mean() >
                                            data_per_month[data_per_month['tariff'] == 'smart']['revenue'].mean()):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

## Вывод: 
#### средняя выручка пользователей тарифа "Ультра" больше средней выручки пользователей тарифа "Смарт"

- Нулевая гипотеза - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов.
- Альтернативная гипотеза - средняя выручка пользователей из Москвы не равна выручке пользователей из других регионов.

In [ ]:
alpha = .05
results_regions = st.ttest_ind (data_per_month[data_per_month['city'] == 'Москва']['revenue'],
                                data_per_month[data_per_month['city'] != 'Москва']['revenue'], equal_var=False)
display(results_regions.pvalue)
if results_regions.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

## Вывод:
#### средняя выручка от пользователей из Москвы равна выручке от пользователей из других регионов

# Общий вывод


### Мы рассмотрели данные пользователей Мегалайна на выборке в 500 человек. Данные оказались корректные, без задвоений и ошибок.

### округлили данные согласно условий тарифа - каждый звонок до минут, а интернет до гб за месяц.
### построили графики по тарифам для пользователей и выявили среднее, медиану, дисперсию и стандартное отклонение
- пользователи тарифа смарт в среднем используют 419 минут при стандартном отклонении 189 минут, в то время как пользователи тарифа Ультра расходуют в среднем 528 минут при стандартном отклонении 306.
- Пользователи Смарт расходуют в среднем 16.3. гб в месяц при стандартном отклонении 5.7. Пользователи Ультра расходуют 19.7 гб траффика при стандартном отклонении 9.7.
- что же касается сообщений, то пользователи на тарифе Смарт тратят в среднем 38.7 сообщений при стандартном распределении 26.8. Пользователи на тарифе Ультра расходуют 61.2 сообщения в среднем при стандартном отклонении 44.7.

### далее мы проверили 2 гипотезы:
1. средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются

для этого мы привели нулевую гипотезу, что выручка от пользователей двух тарифов равна, и альтернативную, что выручка пользователей тарифа Ультра больше, чем выручка пользователей тарифа Смарт. Таким образом мы опровергли нулевую гипотезу и выяснили, что средняя выручка пользователей тарифа "Ультра" больше средней выручки тарифа "Смарт".

2. средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

Здесь мы привели нулевую гипотезу - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов и не смогли ее опровергнуть.

## Таким образом можем сделать вывод, что тариф Ультра с коммерческой точки зрения более выгоден для Мегалайна, тк приносит больше выручки.